# Text classification with the *Longformer*

In a previous [post](https://jesusleal.io/2020/10/20/RoBERTA-Text-Classification/) I explored how to use Hugging Face Transformers Training class to easily create a text classification pipeline. The code was pretty straighforward to implement and I was able to obtain results that put the basic model at a very competitive level. In that post I also briefly discussed one of the main drawbacks of the first iteration of Transformers and BERT based architectures; the sequence lenght is limited to a maximum of 512 characters. The reason behind that limitation is the fact that self-attention mechanism scale quadratically with the input sequence length *O(n^2)*. Given the need to process longer sequences of text a a second generation of attention based models have been proposed. The idea behind this models is to reduce the memory footprint of the attention mechanisms in order to process longer sequences of text; see this really useful [survey of efficient transformers by researchers at Google](https://arxiv.org/pdf/2009.06732.pdf). New models such as the [***Reformer***](https://arxiv.org/pdf/2001.04451.pdf) by Google implement different attention mechanisms such as Local Self Attention, Locality sensitive hashing (LSH) Self-Attention, Chunked Feed Forward Layers, etc. This [post](https://huggingface.co/blog/reformer) from Hugging Face for a detailed discussion). This model can process sequences of half a million tokens with as little as 8GB of RAM. However one big drawback of the model for downstream applications is the fact that the authors did not released pre trained weights of their model and as a time of publication of this post there is no freely available model pretrained on a large corpus. 

Another very promising model, and the subject of this post, is the [***Longformer***](https://arxiv.org/pdf/2004.05150.pdf) by researchers from Allen AI Institure. The Longformer allows the processing sequences of thousand characters without facing the memory bottleneck of BERT like architectures and achieved SOTA at the time of publication in several benchmarks, including text classification. The model achieves an increase in the sequence length through the use of a sliding window attention (each character only attending to characters in +-w) and a global attention token. The authors also propose the use of dilated windows (explain that here). The authors also developed thei own CUDA kernel to increase the efficiency of the matrix multiplication and prevent the complexity to grow. ;. The model is also compatible with the RoBERTA architecture as they use the same tokenizer. 

Given all this nice features I decided to try the model and see how it compares to RoBERTA. One thing that is worth mentioning is that the training and inference speed of the model is significantly slower to RoBERTA. To run it in my modest 1080GTX I had to use several tricks to make it work on my setting such as:
* using FP16 to save memory
* use gradient checkpointing to reuse weights of the graph where possible. The original paper discussing gradient checkpointing can be found [here](https://arxiv.org/pdf/1604.06174.pdf) and a nice [discussion of gradient checkpointing can be hound here](https://qywu.github.io/2019/05/22/explore-gradient-checkpointing.html).
* only process one document at the time and acumulate for 64 steps.

* Explain what the main innovations are. Mention memory scalability issues.
* mention architecture and use of unique kernel
* mentio state of the art and see if this works with imdb to achieve new state of the art
* talk about other stuff


In [1]:
import pandas as pd
import datasets
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments, LongformerConfig
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os

One of the cool features about this model is that you can specify the attention sliding window across different levels. If this parameter is not changed it will assume a default of 512 across all the different layers.

In [2]:
config = LongformerConfig()

config

LongformerConfig {
  "attention_probs_dropout_prob": 0.1,
  "attention_window": 512,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "sep_token_id": 2,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

In [3]:
wandb.login()

wandb: Currently logged in as: jlealtru (use `wandb login --relogin` to force relogin)


True

In [4]:
train_data, test_data = datasets.load_dataset('imdb', split =['train', 'test'], 
                                             cache_dir='/media/data_files/github/website_tutorials/data')

Reusing dataset imdb (/media/data_files/github/website_tutorials/data/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)


The resulting objects contains an [arrow dataset](https://arrow.apache.org/docs/python/dataset.html)a format optimized to work with  all the attributes of the original dataset, including the original text, label, types, number of rows, etc. 

In [5]:
train_data
#dir(train_data)

Dataset(features: {'text': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['neg', 'pos'], names_file=None, id=None)}, num_rows: 25000)

As showed in the previous post we can apply the tokenizer to the train and test subsets using the FastTokenizerFromPretrained class from the Transformers library. To do that we simply define a function that makes a call to the tokenizer class. We can specify if we want to add `padding`, if we want to truncate sentences that are longer than the maximum lenght established, etc. The method returns a `batch_encode` class that holds all the necessary inputs for the model such as `tokens`, `attention_masks`, etc.  We then can use the `map` function and apply the tokenizer function to all the elements of all the splits in dataset.

In [6]:
# load model and tokenizer and define length of the text sequence
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096',
                                                           gradient_checkpointing=False,
                                                           attention_window = 512)
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096', max_length = 1024)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', '

In [7]:
#tokenizer.encode('Mama is not a ft  sentence from training data', return_tensors='pt')
model.config

LongformerConfig {
  "_name_or_path": "allenai/longformer-base-4096",
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "ignore_attention_mask": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 4098,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "sep_token_id": 2,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

In [8]:
# define a function that will tokenize the model, and will return the relevant inputs for the model
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = 'max_length', truncation=True, max_length = 1024)

#attention_mask[:, [0]] = 1

train_data = train_data.map(tokenization, batched = True, batch_size = len(train_data))
test_data = test_data.map(tokenization, batched = True, batch_size = len(test_data))

In [9]:
len(train_data['input_ids'][0])

1024

Once the tokenization process is finished we can use the set the column names and types.

In [10]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

The trainer helper class is designed to facilitate the finetuning of models using the Transformers library. The `Trainer`  class depends on another class called `TrainingArguments` that contains all the attributes to customize the training. `TrainingArguments` contains useful parameter such as output directory to save the state of the model, number of epochs to fine tune a model, use of mixed precision tensors (available with the [Apex](https://github.com/NVIDIA/apex) library), warmup steps, etc. Using the same class we can also ask the model to evaluate the model at the end of each training epoch rather than after a determined amount of steps. To make sure we evaluate at the end of the training epoch we set `evaluation_strategy = 'Epoch'`. For this case we also set the option `load_best_model_at_end` to true, this will guarantee that we will load the best model for evaluation
(according to the metrics defined) at the end of training.

The `Trainer` class provides also allows to implement more sophisticated optmizers and learning rates which can be fed in the `optimizer` option. For this tutorial I use the default gradient descent optimization algorithm provided by the library *AdamW*. AdamW is an optimization based on the original Adam(Adaptive Moment Estimation) that incorporates a regularization term designed to work well with adaptive optimizers; a pretty good discussion of Adam, AdamW and the importance of regularization can be found [here](https://towardsdatascience.com/why-adamw-matters-736223f31b5d). The class also uses a default scheduler to modify the learning rate as the training of the model progresses. The default scheduler on the trainer class is [`get_linear_schedule_with_warmup`](https://huggingface.co/transformers/_modules/transformers/optimization.html#get_linear_schedule_with_warmup) an scheduler that decreases the learning rate linearly until it reaches zero. As mentioned before we can also modify the default values to use a different scheduler. For the learning rate I chose the default of 5e-5 as I wanted to be conservative since this an already pretrained model. Further [Sun et al](https://arxiv.org/pdf/1905.05583.pdf) found that a learning rate of 5e-5 works well for text classification.  I did not modify any of the other parameters of AdamW. 

`Trainer` also makes accumulating gradient steps pretty straightforward. This is relevant when we need to train models on smaller GPU's. For this tutorial I will be using a [GeForce GTX 1080](https://www.nvidia.com/en-sg/geforce/products/10series/geforce-gtx-1080/) that has 8GB of RAM. Given the size of the models (in this case 125 million parameters) and the limitation of the memory.

We can also define if we want to log the training into wanddb. Wandb, short for [Weights and Biasis](https://www.wandb.com/), is a service that allows you visualize the performance of your model and parameters ina very nice dashboad. In this tutorial I assumed you have wandb installed and configured to log the information of weights and parameters. A detailed tutorial of wandb can be found [here](https://wandb.ai/jxmorris12/huggingface-demo/reports/A-Step-by-Step-Guide-to-Tracking-Hugging-Face-Model-Performance--VmlldzoxMDE2MTU). We define the name of the run with `run_name` in the `TrainingArguments` class to easily keep track of the model.

Finally we can also specify the metrics to evaluate the performance of the model on the test set with the `compute_metrics` argument in the `Trainer` class. In this example I selected accuracy, f1 score, precision and recall as suggested in the tutorial by Hugging Face and wrapped them in a functiont hat returns the values for these metrics. This set of metrics provide a very good idea on the performance of the model.

In [11]:
# define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # argmax(pred.predictions, axis=1)
    #pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [12]:
# define the training arguments
training_args = TrainingArguments(
    output_dir = '/media/data_files/github/website_tutorials/results',
    num_train_epochs = 5,
    learning_rate = 3e-5,
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 4,    
    per_device_eval_batch_size= 16,
    evaluation_strategy = "epoch",
    disable_tqdm = False, 
    load_best_model_at_end=True,
    warmup_steps=70,
    weight_decay=0.01,
    logging_steps = 4,
    fp16 = True,
    logging_dir='/media/data_files/github/website_tutorials/logs',
    dataloader_num_workers = 0,
    run_name = 'longformer-classification-updated-rtx3090_paper_replication'
)

In [13]:
# instantiate the trainer class and check for available devices
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [14]:
#import multiprocessing
#multiprocessing.set_start_method('spawn')
#wandb.init()

* mention the use old model
* figure out a wait to save and upload the model 

In [15]:
# train the model
trainer.train()

longformer-classification-updated-rtx3090_paper_replication


wandb: wandb version 0.10.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.1
wandb: Run data is saved locally in wandb/run-20201118_112432-1p5zy9a0
wandb: Syncing run longformer-classification-updated-rtx3090_paper_replication


/home/jlealtru/anaconda3/envs/torch/lib/python3.7/site-packages/datasets/arrow_dataset.py:847: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,0.265537,0.144712,0.946320,0.947124,0.933152,0.961520
1,0.087524,0.153358,0.953120,0.952810,0.959144,0.946560
2,0.007179,0.187394,0.953760,0.954076,0.947601,0.960640
3,0.098312,0.247547,0.950640,0.949821,0.965845,0.934320
4,0.045990,0.252805,0.953840,0.954061,0.949525,0.958640


TrainOutput(global_step=3905, training_loss=0.10297909393505922)

After the training has been completed we can evaluate the performance of the model and make sure we are loading the right model.

In [16]:
#
trainer.save_model('/media/data_files/github/website_tutorials/results/paper_replication')

In [17]:
trainer.evaluate()

{'eval_loss': 0.14471180737018585,
 'eval_accuracy': 0.94632,
 'eval_f1': 0.9471237194641451,
 'eval_precision': 0.9331521739130435,
 'eval_recall': 0.96152,
 'epoch': 4.99968}

The best iteration of our model achieved an accuracy 0.9565, which would put us on [third place](http://nlpprogress.com/english/sentiment_analysis.html) in the leaderboard of sentiment analysis classification with IMDB.

![](images/eval_roberta.svg)

Thats it for this tutorial, hopefully you will find this helpful.